# Testing ETL Process
## Extract pdf and info from intranet site

In [ ]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config

In [ ]:
# Get the url on the config file

url = config()['websites']['alta_patronal']['url']
user = config()['websites']['alta_patronal']['user']
pwd = config()['websites']['alta_patronal']['pwd']
url_stats = config()['websites']['alta_patronal']['url_stats']
date1 = config()['websites']['alta_patronal']['date1']
date2 = config()['websites']['alta_patronal']['date2']
download_dir = config()['websites']['alta_patronal']['download_dir']
path_to_chromedriver = config()['websites']['path_to_chromedriver']


In [ ]:
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import os

options = webdriver.ChromeOptions()

options.add_experimental_option('prefs', {
  'download.prompt_for_download': False,
  'download.default_directory': str(os.path.dirname(os.path.abspath(os.getcwd()))) + download_dir,
  'download.directory_upgrade': True,
  'safebrowsing.enabled': True
})

driver = webdriver.Chrome(executable_path=path_to_chromedriver, options=options)

driver.get(url)

In [ ]:
# Find the fields user and pwd
driver.find_element_by_xpath('//input[@id="cveUsuario"]').get_attribute('id')

In [ ]:
driver.find_element_by_xpath('//input[@id="contrasenia"]').get_attribute('id')

In [ ]:
# Send the credentials
driver.find_element_by_xpath('//input[@id="cveUsuario"]').send_keys(user)

In [ ]:
driver.find_element_by_xpath('//input[@id="contrasenia"]').send_keys(pwd)

In [ ]:
# Locate button to log in
driver.find_element_by_xpath('//input[@type="button"]').get_attribute('name')

In [ ]:
# Log in
driver.find_element_by_xpath('//input[@type="button"]').click()

In [ ]:
# Revisar mensaje de error en login
driver.find_element_by_xpath('//font[@color="red"]/div')

In [ ]:
# Read the error message
driver.find_element_by_xpath('//font[@color="red"]/div').text

In [ ]:
driver.get(url_stats)

In [ ]:
# Locate component date
driver.find_element_by_xpath('//input[@name="fechaInicio"]').get_attribute('id')

In [ ]:
driver.find_element_by_xpath('//input[@name="fechaFin"]').get_attribute('id')

In [ ]:
# Send the first date
driver.find_element_by_xpath('//input[@name="fechaInicio"]').send_keys(date1)

In [ ]:
# Send the second date
driver.find_element_by_xpath('//input[@name="fechaFin"]').send_keys(date2)

In [ ]:
# Now, the next element is a select menu, so we need to import Select from Selenium
from selenium.webdriver.support.select import Select


In [ ]:
# Locate the element
driver.find_element_by_xpath('//select[@id="slcTipoReporte"]').get_attribute('name')

In [ ]:
# Inspect the select and print the options
# First we get the name of the select menu

test = Select(driver.find_element_by_xpath('//select[@id="slcTipoReporte"]'))

#print(test.options[0].text)
#print(test.options[4].text)

In [ ]:
# Select the correct option 
test.select_by_value('4')

In [ ]:
# Locate the button to get the report
driver.find_element_by_xpath('//input[@type="button"]')

In [ ]:
# Click the button
driver.find_element_by_xpath('//input[@type="button"]').click()

In [ ]:
# Close Driver
driver.close()

## Successful file download

In [ ]:
# Install module
!pip install PyPDF2

In [ ]:
import PyPDF2

# Read the PDF
pdfFileObj = open('../downloads/Reporte.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
doc_info = PyPDF2.PdfFileReader(pdfFileObj).getDocumentInfo()

# Show numPages
pdfReader.numPages

In [ ]:
doc_info

In [ ]:
pageObj = pdfReader.getPage(0)

In [ ]:
pageObj.extractText()

In [4]:
# Try using tabula jar
!java -jar ../../tabula/tabula-1.0.4-jar-with-dependencies.jar ../downloads/Reporte.pdf --format CSV --pages all --outfile Reporte_AP_Reingenieria3.csv

In [5]:
import pandas as pd

data_ap = pd.read_csv('Reporte_AP_Reingenieria3.csv', encoding='Windows 1252')
type(data_ap)
data_ap


,REG PATRONAL,MODALIDAD,CIZ,DELEGACIÓN,SUBDELEGACIÓN,HORA,TIEMPO DE TRÁMITE,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,INICIO,TERMINO,HORAS,MINUTOS,SEGUNDOS
1,B8711469,10.0,3.0,14.0,40.0,11/9/20 10:53 AM,11/9/20 11:01 AM,0,8,7
2,C5111713,10.0,1.0,15.0,80.0,11/6/20 9:37 AM,11/6/20 9:40 AM,0,3,25
3,C8411930,10.0,3.0,17.0,13.0,11/9/20 11:16 AM,11/9/20 11:30 AM,0,13,45
4,E3311647,10.0,2.0,25.0,5.0,11/9/20 10:11 AM,11/9/20 10:19 AM,0,8,14
...,...,...,...,...,...,...,...,...,...,...
1734,H0230297,10.0,2.0,34.0,1.0,11/4/20 12:52 PM,11/4/20 12:57 PM,0,4,16
1735,H0230303,10.0,2.0,34.0,1.0,11/6/20 9:41 AM,11/6/20 9:42 AM,0,1,4
1736,H0230304,10.0,2.0,34.0,1.0,11/6/20 10:13 AM,11/6/20 10:19 AM,0,6,34
1737,H0230305,10.0,2.0,34.0,1.0,11/6/20 11:52 AM,11/6/20 11:54 AM,0,2,33
